In [1]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from libs.utils import *
warnings.simplefilter(action='ignore', category=FutureWarning)

import keras
from keras.layers import *
from keras.models import *
from keras.callbacks import *


Using TensorFlow backend.


In [2]:
num_rows = 10000
file_path = './checkpoint/keras_best.h5'

In [3]:
# df, app_cat_cols = application_train_test(num_rows)
# with timer("Process bureau and bureau_balance"):
#     bureau, b_cat_cols = bureau_and_balance(num_rows)
#     print("Bureau df shape:", bureau.shape)
#     df = df.join(bureau, how='left', on='SK_ID_CURR')
#     del bureau
#     gc.collect()

In [4]:
# with timer("Process previous_applications"):
#         prev, prev_cat_cols = previous_applications(num_rows)
#         print("Previous applications df shape:", prev.shape)
#         df = df.join(prev, how='left', on='SK_ID_CURR')
#         del prev
#         gc.collect()

In [5]:
# with timer("Process POS-CASH balance"):
#         pos, pos_cat_cols = pos_cash(num_rows)
#         print("Pos-cash balance df shape:", pos.shape)
#         df = df.join(pos, how='left', on='SK_ID_CURR')
#         del pos
#         gc.collect()

In [6]:
# with timer("Process installments payments"):
#         ins, ins_cat_cols = installments_payments(num_rows)
#         print("Installments payments df shape:", ins.shape)
#         df = df.join(ins, how='left', on='SK_ID_CURR')
#         del ins
#         gc.collect()

In [7]:
# with timer("Process credit card balance"):
#         cc, cc_cat_cols = credit_card_balance(num_rows)
#         print("Credit card balance df shape:", cc.shape)
#         df = df.join(cc, how='left', on='SK_ID_CURR')
#         del cc
#         gc.collect()

In [8]:
# cat_cols = app_cat_cols + b_cat_cols + prev_cat_cols + pos_cat_cols + ins_cat_cols + cc_cat_cols
# for c in cat_cols:
#     print(c, df[c].unique())

In [9]:
# tr_df, te_df, cat_cols = application_train_test(num_rows)
full_df, tr_size, cat_cols = application_train_test(num_rows)

Train samples: 10000, test samples: 10000


In [10]:
y = full_df[:tr_size].TARGET
le = preprocessing.LabelEncoder()
allvalues = y.unique().tolist()
le.fit(allvalues)
y = le.transform(y.values)
# y = to_categorical(y)
cat_len = []
for c in cat_cols:
    cat_len.append(len(full_df[c].unique()))
#cat_cols = [col for col in df.columns if df[col].dtype == 'object']

In [11]:
num_cols = list(set(full_df.columns.tolist()) - set(cat_cols + ['index']))
print(num_cols)

['AMT_INCOME_TOTAL', 'ENTRANCES_MEDI', 'NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER', 'FLOORSMIN_MEDI', 'ELEVATORS_MODE', 'LIVINGAPARTMENTS_MEDI', 'YEARS_BUILD_AVG', 'APARTMENTS_MEDI', 'FLAG_CONT_MOBILE', 'NONLIVINGAREA_MEDI', 'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_DAY', 'ENTRANCES_AVG', 'COMMONAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'AMT_ANNUITY', 'LANDAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'YEARS_BUILD_MODE', 'DAYS_EMPLOYED', 'NEW_CAR_TO_BIRTH_RATIO', 'AMT_GOODS_PRICE', 'TARGET', 'EXT_SOURCE_3', 'NEW_SCORES_STD', 'FLAG_OWN_REALTY', 'FLAG_DOCUMENT_3', 'APARTMENTS_MODE', 'NEW_CREDIT_TO_ANNUITY_RATIO', 'AMT_REQ_CREDIT_BUREAU_QRT', 'COMMONAREA_AVG', 'YEARS_BEGINEXPLUATATION_MEDI', 'NEW_INC_BY_ORG', 'BASEMENTAREA_MEDI', 'NEW_INC_PER_CHLD', 'FLAG_OWN_CAR', 'HOUR_APPR_PROCESS_START', 'LIVINGAREA_MEDI', 'ELEVATORS_MEDI', 'LIVINGAPARTMENTS_AVG', 'ENTRANCES_MODE', 'NEW_DOC_IND_KURT', 'DEF_30_CNT_SOCIAL_CIRCLE', 'NEW_EXT_SOURCES_MEAN', 'LIVINGAREA_AVG', 'DAYS_REGISTRATION', 'REG_CITY_NOT_LIVE_CI

In [12]:
batch_size = 32
import tensorflow as tf
def nn_model(X_num, X_cat , cat_len = cat_len, cat_cols = cat_cols, num_cols = num_cols):
#     cat_out = []
#     for c in cat_cols:
#         cat_input = df[c].copy()
#         in_dim = len(df[c].unique())
#         out_dim = in_dim // 2
#         embed = Embedding(input_dim=in_dim, output_dim = out_dim)
#         out = embed(cat_input)
    input_cat = []
    out_cat = []
    print(X_cat.shape[1])
    for name in cat_cols:
        input_cat.append(Input(shape=(1,), name="cat_" + name))
    for x, c_len in zip(input_cat, cat_len):
        x = Embedding(c_len + 1, c_len // 2, embeddings_initializer ="normal")(x)
        #x = SpatialDropout1D(0.25)(x)
        x = Flatten()(x)
        x = Dense(c_len * 8, activation="relu", kernel_initializer="normal")(x)
        out_cat.append(x)
    out_cat = concatenate(out_cat, axis=-1)
    out_cat = Dense(64, activation="relu", kernel_initializer="normal")(out_cat)
    
    input_num = Input(shape=(X_num.shape[1],), name="numeric")
    out_num = Dense(64, activation="relu", kernel_initializer="normal")(input_num)
    out_num = BatchNormalization()(out_num)
    out_num = Dropout(0.5)(out_num)
    
    out_num_cat = concatenate([out_cat, out_num], axis=-1)
    print(out_num_cat.shape)
    out_num_cat = Dense(64, activation="relu", kernel_initializer="normal")(out_num_cat)
    out_num_cat = BatchNormalization()(out_num_cat)
    print(out_num_cat.shape)
    out_num_cat = Dense(1, activation="sigmoid", kernel_initializer="normal")(out_num_cat)
    print(out_num_cat.shape)
    for i in input_cat:
        print(i)
    model = Model(inputs=[*input_cat, input_num], outputs=out_num_cat)
    return model

In [13]:
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=2, save_best_only=True, save_weights_only=True,
                                 mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=5, min_delta=1e-4)
lr_reduced = ReduceLROnPlateau(monitor='val_loss',
                               factor=0.5,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4,
                               mode='min')

In [14]:
te_df = full_df[tr_size:]
tr_df = full_df[:tr_size]
X_num = np.array(tr_df[num_cols])
X_cat = np.array(tr_df[cat_cols])
model = nn_model(X_num, X_cat)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

19
Instructions for updating:
keep_dims is deprecated, use keepdims instead
(?, 128)
(?, 64)
(?, 1)
Tensor("cat_EMERGENCYSTATE_MODE:0", shape=(?, 1), dtype=float32)
Tensor("cat_FONDKAPREMONT_MODE:0", shape=(?, 1), dtype=float32)
Tensor("cat_HOUSETYPE_MODE:0", shape=(?, 1), dtype=float32)
Tensor("cat_NAME_CONTRACT_TYPE:0", shape=(?, 1), dtype=float32)
Tensor("cat_NAME_EDUCATION_TYPE:0", shape=(?, 1), dtype=float32)
Tensor("cat_NAME_FAMILY_STATUS:0", shape=(?, 1), dtype=float32)
Tensor("cat_NAME_HOUSING_TYPE:0", shape=(?, 1), dtype=float32)
Tensor("cat_NAME_INCOME_TYPE:0", shape=(?, 1), dtype=float32)
Tensor("cat_NAME_TYPE_SUITE:0", shape=(?, 1), dtype=float32)
Tensor("cat_OCCUPATION_TYPE:0", shape=(?, 1), dtype=float32)
Tensor("cat_ORGANIZATION_TYPE:0", shape=(?, 1), dtype=float32)
Tensor("cat_WALLSMATERIAL_MODE:0", shape=(?, 1), dtype=float32)
Tensor("cat_WEEKDAY_APPR_PROCESS_START:0", shape=(?, 1), dtype=float32)
Tensor("cat_FLAG_DOCUMENT_5:0", shape=(?, 1), dtype=float32)
Tensor("cat

In [15]:
model_name = 'keras'
checkpoint_path = './checkpoint'
kfold = KFold(n_splits=5, shuffle=True, random_state=2018)

for fold, (train_index, val_index) in enumerate(kfold.split(y)):

#     model.summary()
    print(f"\n[+] Fold {fold}")

    file_path = f"./checkpoint/keras_{model_name}_best_{fold}.h5"
    checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=2, save_best_only=True,
                                 save_weights_only=True,
                                 mode='min')
    csv_logger = CSVLogger(f'{checkpoint_path}/log_{model_name}_{fold}.csv', append=True, separator=',')
    callbacks_list = [checkpoint, early, lr_reduced, csv_logger]

    X_tr_num =np.array(tr_df[num_cols])[train_index]
    X_tr_cat = [tr_df[c][train_index] for c in cat_cols]
    y_tr = y[train_index]
    
    print(X_tr_num.shape)
    print(len(X_tr_cat))

    X_val_num = np.array(tr_df[num_cols])[val_index]
    X_val_cat = [tr_df[c][val_index] for c in cat_cols]
    y_val = y[val_index]

    history = model.fit([*X_tr_cat, X_tr_num ], y_tr,
                        validation_data=([*X_val_cat, X_val_num], y_val),
                        verbose=1, callbacks=callbacks_list,
                        epochs=10, batch_size=16)


[+] Fold 0
(8000, 106)
19
Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 4s 518us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/10
 384/8000 [>.............................] - ETA: 2s - loss: nan - acc: 0.0000e+00

C:\Users\hungtran\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\callbacks.py:405: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current, self.best):
C:\Users\hungtran\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\callbacks.py:497: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current - self.min_delta, self.best):
C:\Users\hungtran\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\callbacks.py:870: RuntimeWarning: invalid value encountered in less
  self.monitor_op = lambda a, b: np.less(a, b - self.epsilon)


8000/8000 [==============================] - 2s 271us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/10
8000/8000 [==============================] - 2s 271us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 4/10
7904/8000 [============================>.] - ETA: 0s - loss: nan - acc: 0.0000e+00Epoch 00004: val_loss did not improve

Epoch 00004: reducing learning rate to 0.0005000000237487257.
8000/8000 [==============================] - 2s 266us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 5/10
8000/8000 [==============================] - 2s 280us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00

[+] Fold 1
(8000, 106)
19
Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 2s 265us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/10
8000/8000 [==============================] - 2s 270u

In [16]:
len([*X_tr_cat, X_tr_num])

20